In [1]:
#http://stackoverflow.com/questions/32503826/how-can-i-grab-data-series-from-xml-or-tcx-file
#https://github.com/vkurup/python-tcxparser/blob/master/tcxparser.py
#http://www.diveintopython3.net/xml.html
#TODO add wind bearing relative to rider
#imports
from numpy import *
import tcxparser as TCX
import pprint,csv

In [2]:
xmlfile = 'demoRoute.tcx'
data = TCX.TCXParser(xmlfile)
end = len(data.time_values())
samplePoints = 10
ind = linspace(0, (end-1),samplePoints, endpoint=True, retstep=False, dtype=None)
ind = floor(ind)
ind = ind.astype(int)

array([   0,  145,  291,  436,  582,  727,  873, 1018, 1164, 1310])

In [3]:
data.activity.Track.Trackpoint.Position.getchildren()
latitude =data.latitude_points()
longitude = data.longitude_points()
lat = array(latitude)
lat = lat[ix_ (ind)]
long = array(longitude)
long = long[ix_ (ind)]

array([-4.38087  , -4.3313933, -4.31687  , -4.30074  , -4.18785  ,
       -4.17816  , -4.23164  , -4.268    , -4.31112  , -4.38087  ])